<a href="https://colab.research.google.com/github/DonnaVakalis/Livability/blob/master/Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimum Wage Exploration

---

More notes here later...


## Setup



### Import Libraries


In [29]:
# Install pycountry
!pip install pycountry

# Imports and global options

import pandas as pd
from pandas import DataFrame
from matplotlib import pyplot as plt
import zipfile
import os
from google.colab import drive
import requests
from bs4 import BeautifulSoup as bs
import pycountry

import plotly.express as px 

# display numbers with two decimal places
pd.options.display.float_format = '{:,.2f}'.format

# mount google drive
drive.mount('/content/gdrive/') 
# check !ls "/content/gdrive/My Drive/Colab Notebooks/min_wage"

     |████████████████████████████████| 10.1MB 6.8MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=42cb93ddca0fd54b98b61d1dcc295ad587329e8e27fb09170c810665ff395854
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built pycountry
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


### Import Data

Data provenance: 
* Cleaned Min Wage Data [here](https://www.numbeo.com/property-investment/rankings_by_country.jsp?title=2020&displayColumn=6)
* Further wrangling performed by DJW [here](https://danieljwilson.com/datasets/)
* Includes addition by [DJW](https://danieljwilson.com) of Living Wage data [here](https://livingwage.mit.edu/)
* Election data from [MIT Election Lab](https://electionlab.mit.edu/data)

In [ ]:
# import min wage data 
df_min=pd.read_csv("https://danieljwilson.com/datasets/min_wage/min_wage_usa_2020.csv", encoding= 'ISO-8859-1')

# import state codes
df_codes=pd.read_csv("https://danieljwilson.com/datasets/min_wage/USPS_two_letter_state.csv", encoding= 'unicode_escape')

# import election data
df_pol = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/min_wage/1976-2016-president.csv')

In [ ]:
# scrape some data
url = "https://en.wikipedia.org/wiki/List_of_minimum_wages_by_country"
html = requests.get(url).text
soup = bs(html, 'html.parser')
ta=soup.find_all('table',{'class':'wikitable'})

for tn, table in enumerate(ta):

    # preinit list of lists
    rows = table.findAll("tr")
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    cell_n = j + l + m
                    row_n = i + k
                    # in some cases the colspan can overflow the table, in those cases just get the last item
                    cell_n = min(cell_n, len(data[row_n])-1)
                    data[row_n][cell_n] += cell.text
                    print(cell.text)

        data.append(rowD)

df = DataFrame(data,columns=['country_name','notes','annual_USD','annual_GKD','work_hrs_per_wk','per_hour_USD','per_hour_GKD','minwage_as_prct_GDP_percapita','data_year'])
df = df.iloc[2:] # take off header rows
df = df.replace(to_replace= r'\n', value= '', regex=True) # remove backslashes
df_mw = df.copy()

In [71]:
# take out NAs and percentage signs
df_mw  = df_mw .dropna()
df_mw.minwage_as_prct_GDP_percapita  = [[i.replace('%', '') for i in j] for j in df_mw.minwage_as_prct_GDP_percapita]
df_mw.head()

,country_name,notes,annual_USD,annual_GKD,work_hrs_per_wk,per_hour_USD,per_hour_GKD,minwage_as_prct_GDP_percapita,data_year
2,Afghanistan,"5,500 Afghani ($67) per month for non-permanen...","1,047","3,272",40,0.5,1.57,"[1, 6, 8, ., 3, ]",2017
3,Albania,"26,000 Albanian lekë ($240) per month, in priv...","2,418","5,218",40,1.16,2.51,"[4, 5, ., 2, ]",5 May 2017
4,Algeria,"18,000 Algerian dinars ($122.89) per month, na...","1,946","5,622",40,0.94,2.7,"[3, 7, ., 4, ]",1 January 2012
5,Andorra,"€1050.40 per month, €6.06 per hour.[14]","14,243","11,020",40,6.85,5.3,"[2, 8, ]",1 January 2019
6,Angola,"21,454 kwanza ($58) per month; paid thirteen t...","1,681","2,108",44,0.73,0.92,"[3, 2, ., 7, ]",1 April 2019


In [72]:
#Aside: figure out 3-character country codes...

list_countries = df_mw['country_name'].unique().tolist()
# print(list_countries) # Uncomment to see list of countries
d_country_code = {}  # To hold the country names and their ISO
for country in list_countries:
    try:
        country_data = pycountry.countries.search_fuzzy(country)
        # country_data is a list of objects of class pycountry.db.Country
        # The first item  ie at index 0 of list is best fit
        # object of class Country have an alpha_3 attribute
        country_code = country_data[0].alpha_3
        d_country_code.update({country: country_code})
    except:
        print('could not add ISO 3 code for ->', country)
        # If could not find country, make ISO code ' '
        d_country_code.update({country: ' '})
        
 # create a new column iso_alpha in the df
# and fill it with appropriate iso 3 code
for k, v in d_country_code.items():
    df_mw.loc[(df_mw.country_name == k), 'iso_alpha'] = v     

could not add ISO 3 code for ->  Cape Verde
could not add ISO 3 code for ->  Democratic Republic of the Congo
could not add ISO 3 code for ->  North Korea
could not add ISO 3 code for ->  South Korea
could not add ISO 3 code for ->  Laos
could not add ISO 3 code for ->  Northern Cyprus


In [73]:
df = df_mw.copy()
df.per_hour_USD = df.per_hour_USD.astype(float)
df.info()
 

ValueError: ignored

In [46]:


fig = px.choropleth(data_frame = df,
                    locations= "iso_alpha",
                    color= "per_hour_USD",  # value in column  determines color
                    hover_name= "country_name",
                    color_continuous_scale= px.colors.sequential.RdBu)  #  color scale red, yellow green
                    #animation_frame= "Date")

fig.show()



### Clean/Tidy Data

In [ ]:
# Using USPS state codes because Plotly will automatically associate these specific abbreviations with location on the map
df_min = pd.merge(df_min,  df_codes, on='State',how='left') 
df_min.tail()

,State,Wage,lw_10,lw_22,Abbreviation
46,Virginia,7.25,14.00,17.62,VA
47,Washington,13.50,13.47,17.45,WA
48,West Virginia,8.75,10.83,14.76,WV
49,Wisconsin,7.25,11.40,16.49,WI
50,Wyoming,7.25,11.05,16.54,WY


## Exploration 1: 2020 minimum wage by state


### Map of minimum wage

In [ ]:
fig = px.choropleth(df_min,   
                    locations="Abbreviation",  # DataFrame column with locations
                    color="Wage",  # DataFrame column with color values
                    hover_name="State", # DataFrame column hover info
                    locationmode = 'USA-states', # Set to plot as US States
                    color_continuous_scale= px.colors.sequential.RdBu) # Colour palette
fig.update_layout(
    title_text = 'Minimum Wage by State, 2020', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()  # Output the plot to the screen

### Overlay relative minimum wage with other stats: 
* proportion of living wage
* state political party 
* state average age
* Walmarts per capita...

#### proportion of living wage

In [ ]:
# Add column that is minimum wage/living wage
min_df['lw_10_prop'] = (min_df.Wage/min_df.lw_10).round(2)
df_min['prop_lw10'] = (df_min.Wage/df_min.lw_10).round(2)

fig = px.choropleth(df_min,   
                    locations="Abbreviation",  # DataFrame column with locations
                    color="prop_lw10",  # DataFrame column with color values
                    hover_name="State", # DataFrame column hover info
                    locationmode = 'USA-states', # Set to plot as US States
                    color_continuous_scale= px.colors.sequential.gray) # Colour palette
fig.update_layout(
    title_text = 'Proportion Minimum Wage:Living Wage by State, 2020', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()  # Output the plot to the screen

NameError: ignored

#### political party

In [ ]:
df_pol = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/min_wage/1976-2016-president.csv')
df_pol = df_pol[['year','state','state_po','party' ]]
df_pol.info()
df_pol = df_pol.groupby(['year','state_po']).size().reset_index()
df_pol.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3740 entries, 0 to 3739
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      3740 non-null   int64 
 1   state     3740 non-null   object
 2   state_po  3740 non-null   object
 3   party     3404 non-null   object
dtypes: int64(1), object(3)
memory usage: 117.0+ KB


,year,state_po,0
0,1976,AK,4
1,1976,AL,7
2,1976,AR,4
3,1976,AZ,8
4,1976,CA,7


#### Average Age

#### Walmarts per capita

## Exploration 1: 2020 minimum wage by city


In [ ]:
 # libraries
import Basemap
import numpy as np
import matplotlib.pyplot as plt
 
# Make a data frame with the GPS of a few cities:
data = pd.DataFrame({
'lat':[-58, 2, 145, 30.32, -4.03, -73.57, 36.82, -38.5],
'lon':[-34, 49, -38, 59.93, 5.33, 45.52, -1.29, -12.97],
'name':['Buenos Aires', 'Paris', 'melbourne', 'St Petersbourg', 'Abidjan', 'Montreal', 'Nairobi', 'Salvador']
})
 
# A basic map
m=Basemap(llcrnrlon=-160, llcrnrlat=-75,urcrnrlon=160,urcrnrlat=80)
m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
m.drawcoastlines(linewidth=0.1, color="white")
 
# Add a marker per city of the data frame!
m.plot(data['lat'], data['lon'], linestyle='none', marker="o", markersize=16, alpha=0.6, c="orange", markeredgecolor="black", markeredgewidth=1)

 

ModuleNotFoundError: ignored

In [ ]:
fig = px.choropleth(df_min,   
                    locations="Abbreviation",  # DataFrame column with locations
                    color="Wage",  # DataFrame column with color values
                    hover_name="State", # DataFrame column hover info
                    locationmode = 'USA-states', # Set to plot as US States
                    color_continuous_scale= px.colors.sequential.RdBu) # Colour palette
fig.update_layout(
    title_text = 'Minimum Wage by State, 2020', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()  # Output the plot to the screen